In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install gensim
!python -m spacy download es_core_news_sm #No borrar!!! Si no no jala :(

In [4]:
#Paqueting
import pandas as pd
import re
import spacy
import nltk
import csv
import gensim
import numpy as np
from collections import Counter
from fuzzywuzzy import fuzz

In [ ]:
#Cosas en español
nltk.download('wordnet')
nltk.download('omw')
nlp = spacy.load("es_core_news_sm")
#Conectores discursivos
def cargar_conectores(path):
    con_disc = set()
    with open(path, newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            con_disc.update(row)
    return con_disc

con_disc = cargar_conectores("path/conectores_discursivos.csv")
#Más cosas en español
nltk.download('stopwords')
stopword_es = nltk.corpus.stopwords.words('spanish')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
path = 'file.xlsx'
dfo = pd.read_excel(path)

In [157]:
df = dfo.copy() #!!!Para crear y reiniciar los datos

In [ ]:
#Quitar los indicadores con "Otro_Criterio"
otro_c = df[df['TIPO_RELATIVO'] == 'Otro_Criterio']
df = df[df['TIPO_RELATIVO'] != 'Otro_Criterio']

#Para criterio1, criterio2, criterio3

In [ ]:
#Función para limpiar texto
def limpiar(texto):
    if pd.isna(texto):
        return ""
    texto = texto.lower()
    texto = re.sub(r"(100)\b\d+", "", texto)
    texto = re.sub(r"(-1)\b\d+", "", texto)
    texto = re.sub(r"[^\w\s/*-]", "", texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    doc = nlp(texto)
    tokens = [
        token.text for token in doc
        if not token.is_stop
        and token.lemma_ not in con_disc
        and token.text not in stopword_es
    ]
    return " ".join(tokens)


In [ ]:
#Limpieza de texto ára criterio2 que incluye una fracción
def limpiar_c2(texto):
    texto = texto.lower()
    texto = texto.strip()
    matches = re.findall(r"\(([^()]*)\)", texto)
    if matches:
        for match in matches:
            if '/' in match:
                return match
    return texto


In [ ]:
#Función para verificar si un texto es una fracción y además, si el numerador y el denominador son similares
def cociente(text):
    match = re.search(r"(.+?)\s*/\s*(.+)", text)
    if match:
        num = match.group(1).strip()
        den = match.group(2).strip()
        similarity_ratio = fuzz.ratio(num, den)
        return similarity_ratio >= 60
    return False

In [ ]:
#Aquí se aplica la limpieza de texto para criterio1 y criterio2
df['TEXTO_PROCESADO'] = df.apply(lambda row: limpiar(row['TEXTO_COMPARACION1']) if row['CRITERIO'] != 'criterio2' else row['TEXTO_COMPARACION1'], axis=1)

In [ ]:
#Función para comparar el texto procesado previamente con el criterio establecido
#Se buscan palabras particulares y patrones específicos para cada criterio y se indica no coincidencia si se encuentran palabras que no corresponden al criterio.
def comparacion(texto_p, criterio):
    if criterio == 'criterio1':
        pattern = r".*\s*[xX\*]?\s*100" #Patrón de ejemplo
        if (re.search(pattern, texto_p) or re.search(r".*\/.*\\b(palabrac1|palabrac1|palabrac1|palabrac1|multiplicado por 100|x100|x 100|X100|X 100|\*100|\* 100)\\b", texto_p)) and not re.search(r".*\/.*\\b(palabranoc1|palabranoc1|palabranoc1|palabranoc1|palabranoc1|palabranoc1)\\b", texto_p):
            return 'Sí'
    elif criterio == 'criterio2':
        pattern = r'\s*1(?:\s*[xX\*]?\s*100)?' #Patrón de ejemplo
        if cociente(limpiar_c2(texto_p)) and (re.search(pattern, texto_p) or re.search(r".*\/.*\\b(palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|año t -1|t-1|-1*100|-1 * 100|-1 x 100| -1 X 100|t - 1|t- 1|t -1|n-1|n - 1|n -1|n- 1|t1|t 1)\\b",texto_p)):
                return 'Sí'
    elif criterio == 'criterio3':
        pattern = r".*\s*/\s*.*" #Patrón de ejemplo
        if not re.search(pattern, texto_p):
                return 'Sí'
    return 'No'

In [ ]:
#Se crea una columna con la comparación aplicada de la celda anetrior
df['COMPARACION'] = df.apply(lambda row: comparacion(row['TEXTO_PROCESADO'], row['CRITERIO']), axis=1)

In [ ]:
#Esta función compara aquellas entrada que resultaron No en la comparación o tienen como criterio Otros y compara con los otros 3 criterios deifnidos y calcula su similitud, si la hay, la asigna, en caso contrario, devuelve No.
def comparar_otros(row):
    if row['COMPARACION'] == 'No':
        texto_p = row['TEXTO_PROCESADO']
        if (re.search( r".*\s*[xX\*]?\s*100", texto_p) or re.search(r".*\/.*\\b(palabrac1|palabrac1|palabrac1|palabrac1|multiplicado por 100|x100|x 100|X100|X 100|\*100|\* 100)\\b", texto_p))  and not (re.search(r".*\/.*\b(solicitadas|recibidas|requeridas|demandadas|gestionadas|tramitadas)\b", texto_p) and re.search(r'-\s*1.*?[xX\*]?\s*100', texto_p)):
            return 'criterio1'
        elif cociente(texto_p) and (re.search(r'\s*1.*?[xX\*]?\s*100', texto_p) or re.search(r".*\/.*\\b(palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|palabrac2|año t -1|t-1|-1*100|-1 * 100|-1 x 100| -1 X 100|t - 1|t- 1|t -1|n-1|n - 1|n -1|n- 1|t1|t 1)\\b",texto_p)):
            return 'criterio2'
        elif not (re.search(r".*\s*/\s*.*" , texto_p) or (p in texto_p for p in ['cociente', '/', 'entre', 'dividido', 'respecto', 'dividiendo'])):
            return 'criterio3'
    return 'No'

In [141]:
df['OTRA_COINCIDENCIA'] = df.apply(lambda row: comparar_otros(row), axis=1)

In [ ]:
#Salida
df.to_excel("path/Comparación.xlsx")

### Para Otros

Aquí se aplica la función comparar las entradas con criterio Otro con los criterios con patrón definido.

In [ ]:
otro_c['TEXTO_PROCESADO'] = otro_c['TEXTO_COMPARACION1'].apply(limpiar)

In [ ]:
otro_c['COMPARACION'] = otro_c.apply(lambda row: comparar_otros(row['TEXTO_PROCESADO'], row['CRITERIO']), axis=1)

In [ ]:
#Salida
otro_c.to_excel("path/Otros_Comparación.xlsx")